# **Empathetic Generative Model**

**File Structure (Colab)**

Full Path:
/content/gdrive/MyDrive/CS247-Empathy-Mental-Health/EmpDialogue_RecEC

Symlink: /content/CS247-Empathy-Mental-Health/EmpDialogue_RecEC

Output Folder (for all models):
/content/CS247-Empathy-Mental-Health/EmpDialogue_RecEC/outputs

Roberta: /content/CS247-Empathy-Mental-Health/resources/roberta-large.tsv

Glove: /content/CS247-Empathy-Mental-Health/resources/glove.6B.300d.txt

## **Setup**

If running on local device, ignore cell below:

In [ ]:
import os
import shutil

try:

    from google.colab import drive
    drive.mount('/content/gdrive')

    DRIVE_PATH = '/content/gdrive/My\ Drive/CS247-Empathy-Mental-Health'
    DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
    if not os.path.exists(DRIVE_PYTHON_PATH):
      %mkdir $DRIVE_PATH

    ## the space in `My Drive` causes some issues,
    ## make a symlink to avoid this
    # Solved -> symlink for convenience
    SYM_PATH = '/content/CS247-Empathy-Mental-Health'
    if not os.path.exists(SYM_PATH):
      !ln -s $DRIVE_PATH $SYM_PATH

    running_in_colab = True

    # We already mounted in our google drive.
    # Enter the foler where you put files in:
    %cd '/content/CS247-Empathy-Mental-Health'

    # What files are there:
    !ls


except ModuleNotFoundError:
    running_in_colab = False
    print(
        "I guess you are running locally. If you get this message in Colab, check the files."
    )

Clone/update local repo

In [ ]:
if os.path.isdir(SYM_PATH + "/EmpDialogue_RecEC"):
    !git pull
elif not os.path.exists('./EmpDialogue_RecEC'):
    !git clone https://github.com/saunack/EmpDialogue_RecEC.git
    %cd './EmpDialogue_RecEC'
else:
    %cd './EmpDialogue_RecEC'
    !git pull
!pwd

Ensure you're in "CS247-Empathy-Mental-Health/EmpDialogue_RecEC"

In [ ]:
!bash setup.sh

## **Train Emotion Recognition Model**

Outputs the emotion recognition model checkpoint to (ensure it exists):

EmpDialogue_RecEC/outputs/emotion/best_emotion.pt

Emotion data from this model is fed into the generative model

In [ ]:
# Run if emotion recognition model best checkpoint is unavailable
# !bash bash/run_emotion.sh --glove /resources/glove.6B.300d.txt --gpu_id 0
# !cp outputs/emotion/best_emotion.pt models/emotion

## **Train Generative Models**

### **Soft Gate Model**

**Train**

In [ ]:
!bash bash/run_generation.sh \
            --glove "resources/glove.6B.300d.txt" \
            --gpu_id 0 \
            --mode soft \
            --baseline "resources/roberta-large.tsv" \
            --roberta roberta-large \
            --do_train

Checkpoint cell could be finicky. Feel free to change:

In [ ]:
# Change checkpoint based on what checkpoint you want to use
checkpoint = "checkpoint_other_131.pt"

source_path = f"{checkpoint}"
destination_path = "outputs/soft_gen/checkpoint_best.pt"

# Copy the file from source to destination
shutil.copyfile(source_path, destination_path)

**Test**

In [ ]:
!bash bash/run_generation.sh \
            --glove "resources/glove.6B.300d.txt" \
            --gpu_id 0 \
            --mode soft \
            --roberta roberta-large \
            --baseline "resources/roberta-large.tsv" \
            --do_test

### **Hard Gate Model**

**Train**

In [ ]:
!bash bash/run_generation.sh \
            --glove "resources/glove.6B.300d.txt" \
            --gpu_id 0 \
            --mode hard \
            --baseline "resources/roberta-large.tsv" \
            --roberta roberta-large \
            --do_train

Checkpoint cell could be finicky. Feel free to change:

In [ ]:
# Change checkpoint based on what checkpoint you want to use
checkpoint = "checkpoint_other_131.pt"

source_path = f"/content/CS247-Empathy-Mental-Health/{checkpoint}"
destination_path = "/content/CS247-Empathy-Mental-Health/EmpDialogue_RecEC/outputs/hard_gen/checkpoint_best.pt"

# Copy the file from source to destination
shutil.copyfile(source_path, destination_path)

**Test**

In [ ]:
9!bash bash/run_generation.sh \
             --glove "/resources/glove.6B.300d.txt" \
             --gpu_id 0 \
             --mode hard \
             --roberta roberta-large \
             --baseline "resources/roberta-large.tsv" \
             --do_test